In [1]:
from iism_lab2 import a5_generator
import numpy as np


def get_a5_generator():
    S19 = np.random.randint(2, size=19)
    S22 = np.random.randint(2, size=22)
    S23 = np.random.randint(2, size=23)
    gen = a5_generator(S19, S22, S23)
    return gen


def uniform_psi(gen, n=10):
    while True:
        yield (int(''.join([str(next(gen)) for _ in range(n)]), 2) + 1) / (2 ** n + 1)

$$\int_{88}^{99} \ln{(x)}\cdot\sin{(x)} dx = \int_{88}^{99} \frac{1}{11} \cdot 11 \cdot \ln{(x)}\cdot \sin{(x)} = E\{11\cdot\ln{(x)}\cdot\sin{(x)}\}\approx \frac{1}{n}\sum_{i=1}^{n} 11 \cdot \ln{(x_i)}\sin{(x_i)}$$

In [2]:
psi = uniform_psi(get_a5_generator())
exps = np.array([11 * next(psi) + 88 for _ in range(2000)])
print(np.mean(exps))

93.5477292683


$4.2818$

In [3]:
value = np.mean(11 * np.log(exps) * np.sin(exps))
print(value)

4.64931498953


In [4]:
exps = np.random.uniform(low=88, high=99, size=100000)
value = np.mean(11 * np.log(exps) * np.sin(exps))
print(value)

4.35111202721


Теперь решим $$\iint\limits_{|x| + |y| < 1}\frac{x^3+3xy}{e^{-y}}dxdy = \iint\limits_{|x| + |y| < 1} \frac{1}{2} \cdot 2 \cdot \frac{x^3 + 3xy}{e^{-y}}dxdy \approx \frac{1}{n} \sum\limits_{i=1}^{n} 2 \cdot \frac{x_i^3 + 3x_iy_i}{e^{-y_i}}$$

Правильное значение: $0$

In [5]:
exps = np.random.uniform(low=-1/2**0.5, high=1/2**0.5, size=(10000, 2))
M = np.array([[np.cos(np.pi / 4), -np.sin(np.pi / 4)], 
              [np.sin(np.pi / 4), np.cos(np.pi / 4)]])
for i in range(len(exps)):
    exps[i] = (M.dot(exps[i].reshape((2, 1)))).reshape((1, 2))
x, y = exps.reshape((2, 10000))
value = np.mean(2 * (x ** 3 + 3 * x * y) / np.exp(-y))
print(value)

-0.0306598626965


Решим систему $x = Ax + f$ методом Монте-Карло

In [6]:
data = np.genfromtxt('data/Юнусов.txt', delimiter=' ')
A, f = data[:-1], data[-1]
print(A.shape, f.shape)

(37, 37) (37,)


Как видим, все собственные значения матрицы меньше 1

In [7]:
np.max(A)

0.0576739

In [8]:
from copy import deepcopy
x_new = deepcopy(f)
x_prev = deepcopy(f) - 1000
while abs(np.max(x_prev - x_new)) > 0.00001:
    x_prev = deepcopy(x_new)
    x_new = A.dot(x_new) + f
    print(np.max(x_new - A.dot(x_new) - f))

-0.00875601503225
-0.0116750194937
-0.00238197167137
-0.000603520518509
-0.000163411487571
-4.45742467821e-05
-1.21214950313e-05
-3.27321585059e-06
-8.8382187885e-07


In [9]:
N, m = 500, 10000
H = np.eye(37)

result = []
for l in range(37):
    Q, ksi = [0.] * (N + 1), [0.] * m
    for j in range(m):
        i = [np.random.randint(low=0, high=36) 
             for _ in range(N + 1)]
        Q[0] = H[l][i[0]] * 37
        for k in range(1, N + 1):
            Q[k] = Q[k - 1] * A[i[k - 1]][i[k]] * 37
        for k in range(N + 1):
            ksi[j] += Q[k] * f[i[k]]
    result.append(np.mean(ksi))

result = np.array(result)
print(result)
print(np.max(result - A.dot(result) - f))

[ 9.10148361  0.63205519  9.70296316  6.70254425  2.54242549  2.48866104
  4.13577141  7.62291524  9.72992637  6.20926665  4.93518637  6.38417904
  2.05305854  7.34992873  0.76334514  0.76531959  0.69617921  0.74861612
  6.8089143   7.1058568   9.72365022  7.90979749  3.39826224  2.37760994
  7.63332616  2.31439282  7.55997939  4.26558595  9.42629839  6.49872663
  5.99626742  4.86658138  7.15900301  0.07736657  0.69906862  2.90220351
  0.        ]
0.713502879248
